In [8]:
import os
import rasterio
from rasterio.warp import calculate_default_transform, Resampling
from rasterio.enums import Resampling

# Definir la proyección de destino
target_crs = 'EPSG:25830'

parque = 'Teide'

# Ruta a la carpeta que contiene los archivos raster
folder_path = 'C:/Users/carlo/Desktop/Biophysical_variables(bio) - copia/Teide/lulc.tif_mw1'



# Obtener una lista de todos los archivos en la carpeta
files_in_folder = os.listdir(folder_path)

# Filtrar solo los archivos raster
raster_files = [file_name for file_name in files_in_folder if file_name.endswith('.tif') or file_name.endswith('.tiff')]

for file_name in raster_files:
    # Ruta completa al archivo raster
    raster_path = os.path.join(folder_path, file_name)

    # Abrir el archivo raster
    with rasterio.open(raster_path) as src:
        # Obtener información sobre la transformación y las dimensiones del raster
        transform, width, height = calculate_default_transform(
            src.crs, target_crs, src.width, src.height, *src.bounds)

        # Definir los parámetros para escribir el archivo raster con la nueva proyección
        kwargs = src.meta.copy()
        kwargs.update({
            'crs': target_crs,
            'transform': transform,
            'width': width,
            'height': height
        })

        # Ruta para guardar el archivo raster con la nueva proyección
        output_path = os.path.join(folder_path, 'reproyectado_' + file_name)

        # Crear el archivo raster reproyectado
        with rasterio.open(output_path, 'w', **kwargs) as dst:
            for i in range(1, src.count + 1):
                rasterio.warp.reproject(
                    source=rasterio.band(src, i),
                    destination=rasterio.band(dst, i),
                    src_transform=src.transform,
                    src_crs=src.crs,
                    dst_transform=transform,
                    dst_crs=target_crs,
                    resampling=Resampling.nearest)


CRSError: Cannot find coordinate operations from 'ENGCRS["unnamed",EDATUM[""],CS[Cartesian,2],AXIS["easting",east,ORDER[1],LENGTHUNIT["metre",1,ID["EPSG",9001]]],AXIS["northing",north,ORDER[2],LENGTHUNIT["metre",1,ID["EPSG",9001]]]]' to 'EPSG:25830'

In [1]:
import os
import rasterio
from rasterio.warp import calculate_default_transform, Resampling
from rasterio.enums import Resampling
from rasterio.crs import CRS

# Definir la proyección de destino
target_crs = CRS.from_epsg(25830)

parque = 'Teide'

# Ruta a la carpeta que contiene los archivos raster
folder_path = f'C:/Users/carlo/Desktop/borrar/{parque}/Landscape'

# Obtener una lista de todos los archivos en la carpeta
files_in_folder = os.listdir(folder_path)

# Filtrar solo los archivos raster
raster_files = [file_name for file_name in files_in_folder if file_name.endswith('.tif') or file_name.endswith('.tiff')]

for file_name in raster_files:
    # Ruta completa al archivo raster
    raster_path = os.path.join(folder_path, file_name)

    # Abrir el archivo raster
    with rasterio.open(raster_path) as src:
        # Obtener información sobre la transformación y las dimensiones del raster
        transform, width, height = calculate_default_transform(
            src.crs, target_crs, src.width, src.height, *src.bounds)

        # Definir los parámetros para escribir el archivo raster con la nueva proyección
        kwargs = src.meta.copy()
        kwargs.update({
            'crs': target_crs,
            'transform': transform,
            'width': width,
            'height': height
        })

        # Ruta para guardar el archivo raster con la nueva proyección
        output_path = os.path.join(folder_path, 'reproyectado_' + file_name)

        # Crear el archivo raster reproyectado
        with rasterio.open(output_path, 'w', **kwargs) as dst:
            for i in range(1, src.count + 1):
                rasterio.warp.reproject(
                    source=rasterio.band(src, i),
                    destination=rasterio.band(dst, i),
                    src_transform=src.transform,
                    src_crs=src.crs,
                    dst_transform=transform,
                    dst_crs=target_crs,
                    resampling=Resampling.nearest)


CRSError: Cannot find coordinate operations from 'ENGCRS["unnamed",EDATUM[""],CS[Cartesian,2],AXIS["easting",east,ORDER[1],LENGTHUNIT["metre",1,ID["EPSG",9001]]],AXIS["northing",north,ORDER[2],LENGTHUNIT["metre",1,ID["EPSG",9001]]]]' to 'EPSG:25830'

In [9]:
import os
from osgeo import gdal

parque = 'Teide'

def reproyectar_raster(archivo_entrada, archivo_salida, crs):
    try:
        # Abrir el raster de entrada
        raster_entrada = gdal.Open(archivo_entrada)

        # Reproyectar el raster
        gdal.Warp(archivo_salida, raster_entrada, dstSRS=crs)

        # Cerrar los datasets
        raster_entrada = None

        print(f"Reproyección aplicada a {archivo_entrada}")
    except Exception as e:
        print(f"Error al reproyectar {archivo_entrada}: {str(e)}")

# Definir la carpeta de entrada y salida
parque = "Teide"  # Asegúrate de definir el nombre del parque correctamente
carpeta_entrada = 'C:/Users/carlo/Desktop/Biophysical_variables(bio) - copia/Teide/lulc.tif_mw1'
carpeta_salida = f'C:/Users/carlo/Desktop/borrar/{parque}/Landscape/test'

# Definir la proyección de destino (EPSG 25830)
crs = 'EPSG:25830'

# Verificar si la carpeta de salida existe y crearla si no
if not os.path.exists(carpeta_salida):
    os.makedirs(carpeta_salida)
    print(f"Carpeta de salida creada en {carpeta_salida}")

# Iterar sobre cada archivo raster en la carpeta de entrada
for archivo in os.listdir(carpeta_entrada):
    if archivo.endswith('.tif'):
        # Construir las rutas de archivo de entrada y salida
        archivo_entrada = os.path.join(carpeta_entrada, archivo)
        archivo_salida = os.path.join(carpeta_salida, archivo)
        
        # Reproyectar el archivo raster
        reproyectar_raster(archivo_entrada, archivo_salida, crs)

print("Proceso completado.")




Reproyección aplicada a C:/Users/carlo/Desktop/Biophysical_variables(bio) - copia/Teide/lulc.tif_mw1\ai.tif
Reproyección aplicada a C:/Users/carlo/Desktop/Biophysical_variables(bio) - copia/Teide/lulc.tif_mw1\circle_am.tif
Reproyección aplicada a C:/Users/carlo/Desktop/Biophysical_variables(bio) - copia/Teide/lulc.tif_mw1\circle_cv.tif
Reproyección aplicada a C:/Users/carlo/Desktop/Biophysical_variables(bio) - copia/Teide/lulc.tif_mw1\circle_md.tif
Reproyección aplicada a C:/Users/carlo/Desktop/Biophysical_variables(bio) - copia/Teide/lulc.tif_mw1\circle_mn.tif
Reproyección aplicada a C:/Users/carlo/Desktop/Biophysical_variables(bio) - copia/Teide/lulc.tif_mw1\circle_ra.tif
Reproyección aplicada a C:/Users/carlo/Desktop/Biophysical_variables(bio) - copia/Teide/lulc.tif_mw1\circle_sd.tif
Reproyección aplicada a C:/Users/carlo/Desktop/Biophysical_variables(bio) - copia/Teide/lulc.tif_mw1\contag.tif
Reproyección aplicada a C:/Users/carlo/Desktop/Biophysical_variables(bio) - copia/Teide/lu

In [6]:
import os
import shutil

# Lista de nombres de parques

#// Aiguestortes
#// Ordesa
#// Peneda
#// Guadarrama
#// Picos
#// SierraNieves
#// SierraNevada
#// Teide
parques = ['Aiguestortes', 'Ordesa', 'Peneda', 'Guadarrama', 'Picos', 'SierraNieves', 'SierraNevada','Teide']

# Lista de nombres de archivos TIFF
nombres_archivos_tiff = [
    "ai",
    "circle_mn",
    "contag",
    "contig_mn",
    "ed",
    "enn_mn",
    "frac_mn",
    "iji",
    "lsi",
    "msidi",
    "msiei",
    "np",
    "para_mn",
    "pd",
    "pr",
    "shape_mn",
    "shdi",
    "shei",
    "sidi",
    "siei",
    "ta"
]

# Iterar sobre cada parque
for parque in parques:
    # Directorio de origen y destino para el parque actual
    directorio_origen = f"C:/Users/carlo/Desktop/Biophysical_variables(bio) - copia/{parque}/lulc.tif_mw1"
    directorio_destino = f"C:/Users/carlo/Desktop/borrar/{parque}/Landscape"
    
    # Iterar sobre los archivos en el directorio de origen
    for archivo in os.listdir(directorio_origen):
        nombre, extension = os.path.splitext(archivo)
        # Verificar si la extensión es .tif y el nombre está en la lista
        if extension.lower() == ".tif" and nombre in nombres_archivos_tiff:
            # Construir la ruta completa del archivo de origen y destino
            ruta_origen = os.path.join(directorio_origen, archivo)
            ruta_destino = os.path.join(directorio_destino, archivo)
            # Mover el archivo a la carpeta de destino
            shutil.move(ruta_origen, ruta_destino)



In [14]:
#// Aiguestortes
#// Ordesa
#// Peneda
#// Guadarrama
#// Picos
#// SierraNieves
#// SierraNevada
#// Teide


parque = 'SierraNieves'

# Directorio de origen y destino
directorio_origen = f"C:/Users/carlo/Desktop/Biophysical_variables(bio) - copia/{parque}/lulc.tif_mw1"
directorio_destino = f"C:/Users/carlo/Desktop/borrar/{parque}/Landscape"

# Lista de nombres de archivos TIFF
nombres_archivos_tiff = [
    "ai",
    "circle_mn",
    "contag",
    "contig_mn",
    "ed",
    "enn_mn",
    "frac_mn",
    "iji",
    "lsi",
    "msidi",
    "msiei",
    "np",
    "para_mn",
    "pd",
    "pr",
    "shape_mn",
    "shdi",
    "shei",
    "sidi",
    "siei",
    "ta"
]

# Iterar sobre los archivos en el directorio de origen
for archivo in os.listdir(directorio_origen):
    nombre, extension = os.path.splitext(archivo)
    # Verificar si la extensión es .tiff y el nombre está en la lista
    if extension.lower() == ".tif" and nombre in nombres_archivos_tiff:
        # Construir la ruta completa del archivo de origen y destino
        ruta_origen = os.path.join(directorio_origen, archivo)
        ruta_destino = os.path.join(directorio_destino, archivo)
        # Mover el archivo a la carpeta de destino
        shutil.copy(ruta_origen, ruta_destino)